# Import thư viện

In [ ]:
# Thư viện cơ bản
import pandas as pd
import numpy as np


# Thư viện vẽ biểu đồ
import matplotlib.pyplot as plt
import seaborn as sns

# Bỏ qua các cảnh báo
import warnings
warnings.filterwarnings('ignore')

# Thư viện tiền xử lý dữ liệu
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder, label_binarize, RobustScaler
from imblearn.over_sampling import SMOTE
from sklearn.multiclass import OneVsRestClassifier

# Thư viện mô hình
from sklearn.tree import DecisionTreeClassifier, plot_tree

# Thư viện đánh giá mô hình
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, roc_curve, auc, confusion_matrix

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
# Tập train chiếm 75% dữ liệu
train.shape

In [ ]:
test.head()

In [ ]:
# Tập test chiếm 25% dữ liệu
test.shape

# Tiền xử lý tập train

In [ ]:
train.describe().T

In [ ]:
train['Response'].value_counts()

In [ ]:
train['Response'].value_counts() / len(train) * 100

In [ ]:
# Biểu đồ phân phối nhãn
sns.countplot(x = train['Response'])

# Nhãn phân phối khá lệch

In [ ]:
# Gộp nhãn mới như sau:
# Rủi ro thấp: 1, 2, 3, 4 = 0
# Rủi ro trung bình: 5, 6, 7 = 1
# Rủi ro cao: 8 = 2
def map_response(x):
    if x in [1, 2, 3, 4]:
        return 0
    elif x in [5, 6, 7]:
        return 1
    else:
        return 2
train['Response'] = train['Response'].map(map_response)

In [ ]:
train['Response'].value_counts()

In [ ]:
train['Response'].value_counts() / len(train) * 100

In [ ]:
# Nhãn đã được phân phối lại
sns.countplot(x = train['Response'])

In [ ]:
le = LabelEncoder()
# Chuyển cột định tính thành định lượng
train['Product_Info_2'] = le.fit_transform(train['Product_Info_2'])

In [ ]:
# Xem các cột null trong tập train
train_null_cols = train.isnull().sum()[train.isnull().sum() > 0]
train_null_cols

In [ ]:
# Xem tỉ lệ các cột null trong tập train
train_null_cols / len(train) * 100

In [ ]:
# Drop tất cả cột có tỉ lệ null >= 50%
train = train.loc[:, train.isnull().mean() < 0.5]

In [ ]:
# Fill null với trung vị
train = train.fillna(train.median())

# Tiền xử lý tập test

In [ ]:
test.describe().T

In [ ]:
# Chuyển cột định tính thành định lượng với thông tin đã huấn luyện từ tập train
test['Product_Info_2'] = le.transform(test['Product_Info_2'])

In [ ]:
# Xem các cột null trong tập test
test_null_cols = test.isnull().sum()[test.isnull().sum() > 0]
test_null_cols

In [ ]:
# Xem tỉ lệ các cột null trong tập test
test_null_cols / len(test) * 100

In [ ]:
# Drop tất cả cột có tỉ lệ null >= 50%
test = test.loc[:, test.isnull().mean() < 0.5]

In [ ]:
# Fill null với trung vị
test = test.fillna(test.median())

# Chia Train, Validation

- Lí do chia tập dữ liệu train thành train và valid là do tập test không có nhãn
- => phải huấn luyện và kiểm tra trên train và valid sau đó đưa dự đoán vào test

In [ ]:
# Tách dữ liệu: X - thuộc tính | y - nhãn
X = train.drop(['Response'], axis=1)
y = train['Response']

In [ ]:
# Sau khi chia dữ liệu: Train - 60%, Validation - 15%, Test - 25%
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=1/5, random_state=42, stratify=y)
# Khi này X_test chính là tập test
X_test = test

In [ ]:
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
robust = RobustScaler()
X_train = robust.fit_transform(X_train)
X_val = robust.transform(X_val)
X_test = robust.transform(X_test)

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape

# Chạy mô hình

- Lựa chọn mô hình Decision Tree 
## Lí do:
+ Dễ trực quan hóa bằng sơ đồ cây => dễ giải thích
+ Không cần scale 
+ Không sợ ngoại lai vì chia nhánh dựa trên độ hỗn loạn và độ tăng thông tin
+ Chống overfitting dễ dàng bằng cách điều chỉnh độ sâu tối đa, số lá tối thiểu, ...

In [ ]:
clf_dt = DecisionTreeClassifier(random_state=42, class_weight='balanced')

# Huấn luyện mô hình
clf_dt.fit(X_train, y_train)

In [ ]:
# Dự đoán trên tập validation
y_pred = clf_dt.predict(X_val)

In [ ]:
# Ma trận nhầm lẫn
ConfusionMatrixDisplay.from_estimator(clf_dt, X_val, y_val, cmap='Blues', colorbar=False)

## Cách đọc ma trận nhầm lẫn có nhiều hơn 2 nhãn:
+ Đường chéo chính: True Positive
+ Các hàng trừ ô nằm trên đường chéo chính: FN
+ Các cột trừ ô nằm trên đường chéo chính: FP

Ví dụ minh họa:
+ Ma trận có kích thước 3x3
+ Ở đây TP của nhãn 0 là 1054 (vị trí 0,0)
+ FP: Tổng của cột 0 trừ vị trí (0,0) => tổng từ (1,0) đến (2,0)
+ FN: Tổng của hàng 0 trừ vị trí (0,0) => tổng từ (0,1) đến (0,2)

In [ ]:
print(classification_report(y_val, y_pred))

In [ ]:
# # Vẽ cây (chỉ vẽ 10 tầng đầu tiên)
# plt.figure(figsize=(20,10))
# plot_tree(clf_dt, max_depth=10, feature_names=train.columns[:-1], filled=True)
# plt.show()

# # Cây chưa tối ưu

# Tối ưu mô hình

## Dùng GridSearchCV để tìm parameter
Lí do:
- Tối ưu toàn diện nhiều yếu tố: max_depth, min_samples_split, min_samples_leaf, max_leaf_nodes, ...
- Các tham số trên trực tiếp ảnh hưởng đến độ sâu và độ phức tạp => tránh overfit/underfit
- Có thể kết hợp cross-validation
- Hợp với dữ liệu nhiều feature

In [ ]:
param_grid = {
    'criterion': ['gini','entropy'],                       # Tiêu chí chia nhánh
    'max_depth': [10, 20],                        # Giới hạn độ sâu
    'min_samples_split': [2, 5, 10],                        # Số mẫu tối thiểu để split
    'min_samples_leaf': [5, 10],                            # Số mẫu tối thiểu trong mỗi lá
    'max_features': ['sqrt', 'log2'],                       # Số lượng thuộc tính tối đa để chia nhánh
    'class_weight': [None, 'balanced']                      # Cân bằng nhãn trong loss
}

In [ ]:
grid_search = GridSearchCV(
    estimator=clf_dt,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train, y_train)

# f1_macro: Tính toán F1-score cho mỗi lớp, sau đó tính trung bình của tất cả các lớp mà không tính trọng số (các lớp có số mẫu nhiều hơn sẽ không ảnh hưởng nhiều hơn các lớp ít mẫu).

In [ ]:
best_param = grid_search.best_params_

In [ ]:
best_estimator = grid_search.best_estimator_

In [ ]:
best_dt = DecisionTreeClassifier(**best_param, random_state=42)

In [ ]:
y_pred_best = best_estimator.predict(X_val)

In [ ]:
cm = confusion_matrix(y_val, y_pred_best)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_estimator.classes_)

plt.figure(figsize=(8,6))
disp.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plot_tree(best_estimator, filled=True, feature_names=train.columns[:-1], class_names=['0', '1', '2'], rounded=True)
plt.show()